W zadaniu tym użyjemy algorytmu random forest do stworzenia klasyfikatora rozróżniającego cyfry ze zbioru MNIST. 

**Przygotowanie danych**

In [1]:
import tensorflow as tf

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train_flat = x_train.reshape(-1, 28 * 28).astype('float32')
x_test_flat = x_test.reshape(-1, 28 * 28).astype('float32')

x_train_flat /= 255.0
x_test_flat /= 255.0

2025-04-03 20:15:36.101461: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-03 20:15:36.107015: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-03 20:15:36.129076: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743704136.156400   10738 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743704136.164324   10738 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743704136.184382   10738 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

**Random Forest** to algorytm uczenia maszynowego, którego ideą jest stworzenie zbioru **drzew decyzyjnych**, które uczą się niezależnie na rozłącznych podzbiorach danych.

**Drzewo decyzyjne** to model uczenia maszynowego, który iteracyjnie dokonuje klasyfikacji danych na podstawie wybranych cech. Podziały wybierane są chierarchicznie minimalizując zanieczyszczenie podzbiorów poprzez, np. maksymalizację wskaźnika Gini'ego wyrażanego wzorem: <br>
$G = 1 - \sum_{i=1}^{n} p_i^2$


gdzie:
- $n$ – liczba klas w zbiorze danych,
- $p_i$ – prawdopodobieństwo, że losowo wybrana próbka należy do klasy $i$.

Dane treningowe dzielone są na losowe próbki danych w procesie tzw. **bootstrap aggregation**, który zapobiega przeuczeniu modelu, czyli sytuacji, w której model świetnie działa na danych treningowych lecz zawodzi po dostarczeniu danych spoza tego zbioru. <br>

Zaproponowana struktura lasu losowego została wyznaczona poprzez testowanie algorytmu random forest na kombinacjach parametrów:
- n_estimators = [100, 200, 300] (liczba drzew w lesie losowym)
- max_depth = [10, 20, 30] (maksymalna głębokość drzewa decyzyjnego)
- min_samples_split = [2, 5, 10] (minimalna wielkość zbioru w węźle, dla której możliwy jest podział).

Do wyznaczenia najlepszej kombinacji parametrów została użyta walidacja krzyżowa (cross-validation), czyli wielokrotne dzielenie danych na zestawy treningowe i testowe, aby model mógł być oceniany na różnych częściach tych danych. Ziarno generatora liczb losowych ustawione jest na wartość 66.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

rf_clf = RandomForestClassifier(random_state=66)

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(estimator=rf_clf, param_grid=param_grid, cv=3, n_jobs=-1)
grid_search.fit(x_train_flat, y_train)

print(f"Best Parameters: {grid_search.best_params_}")

best_rf_clf = grid_search.best_estimator_

y_pred = best_rf_clf.predict(x_test_flat)

accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy with Best Parameters: {accuracy * 100:.2f}%")

Najskuteczniejsza okazała się kombinacja n_estimators = 300, max_depth = 20, min_samples_split = 2.

In [18]:
clf = RandomForestClassifier(max_depth=20, min_samples_split=2, n_estimators=300, random_state=66)
clf.fit(x_train_flat, y_train)

RandomForestClassifier(max_depth=20, n_estimators=300, random_state=66)

Predykcja klasy zadanego wejścia polega na jej predykcji przez każde z wytrenowanych drzew, a ostateczna decyzja podejmowana jest większością głosów.

In [19]:
y_pred = clf.predict(x_test_flat)

In [20]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98       980
           1       0.99      0.99      0.99      1135
           2       0.96      0.97      0.97      1032
           3       0.96      0.96      0.96      1010
           4       0.97      0.97      0.97       982
           5       0.97      0.96      0.97       892
           6       0.98      0.98      0.98       958
           7       0.97      0.96      0.97      1028
           8       0.96      0.96      0.96       974
           9       0.96      0.95      0.95      1009

    accuracy                           0.97     10000
   macro avg       0.97      0.97      0.97     10000
weighted avg       0.97      0.97      0.97     10000



**Dokładność**: 97% <br>
**Czułość**: największą czułością charakteryzują się klasy 0 oraz 1, dla których utrzymuje się ona na poziomie 99%. Najsłabiej poradziła sobie klasa 9, dla której wynosi ona 95%.<br>
**Precyzja**: w ogólności precyzja utrzymuje się w okolicach 96-97%. Dobrze poradziły sobie jednak klasy 1 oraz 6, które charateryzują się precyzją odpowiednio 99% oraz 98%.<br>

Zarówno dokładność, średnia czułość oraz średnia precyzja utrzymują się na poziomie 97%, co świadczy o skuteczności użycia algorytmu Random Forest z parametrami n_estimators = 300, max_depth = 20, min_samples_split = 2.